In [6]:
import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import json
import datetime as dt
import warnings
import duckdb as db



In [7]:
# Set display options

pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

cxn = db.connect('./DB/dbfile')

In [8]:
# Parse Secrets

try:
    secrets = json.load(open(file='./secrets.json'))
    print('Secrets file found.')
except:
    print('Secrets file not found or some error in file. Make sure the file exists in the same directory as the code.')

if secrets:
    refresh_token = secrets['refresh_token']

    app_id = secrets['app_id']

    app_secret = secrets['app_secret']

    request_body = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
            "scope": "openid",
            "resource": "https://api.partnercenter.microsoft.com",
            "client_id": app_id,
            "client_secret": app_secret,
        }

    base_url = (
    'https://api.partnercenter.microsoft.com'
    )


    isAccessTokenObtained = 0


Secrets file found.


In [9]:
def query(sqlString:str)->None:
    try:
        return cxn.query(sqlString).show(max_width = 100000, max_rows = 100000)
    except Exception as e:
        return e

In [10]:
# Get access token.

try:
    req = rs.post(
                "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                ,data = request_body)
    
    access_token = json.loads(req.text)['access_token']
    isAccessTokenObtained = 1
    print('Refresh Token valid, access token obtained. \nAccess token : ',access_token[:20]+'...')
except:
    print('Refresh token expired, or no internet connection. \nUnable to obtain access token')
    print(json.loads(req.text))


# Get invoices.

if isAccessTokenObtained == 1:
        
        HTTPheaders = {'Authorization': 'Bearer ' + access_token}

        relativeInvoicesURL = '/v1/invoices'

        invoicesResponse = json.loads(
        rs.get(
                f"{base_url}{relativeInvoicesURL}",
                headers=HTTPheaders)
        .content)


        invoicesDF = pd.DataFrame(invoicesResponse['items'])
        
        
        


Refresh Token valid, access token obtained. 
Access token :  eyJ0eXAiOiJKV1QiLCJh...


In [17]:
cxn.close()

In [14]:
r = rs.get("https://api.partnercenter.microsoft.com/v1/invoices/T000001234/lineitems?provider=onetime&invoiceLineItemType=usagelineitems&currencyCode=usd&period=previous&size=2000")
r

<Response [401]>

In [ ]:
writeInvoiceToFile('G038562576','January_2024')

January_2024 file written to onetime_invoices folder 



In [ ]:
def writeInvoiceToFile(invoiceID:str, invoiceMonth:str)->None:

    response = \
    json.loads(
    rs.get(
            f"{base_url}/v1/invoices/Recurring-{invoiceID}/lineitems/Office/BillingLineItems",
            headers=HTTPheaders)
    .content) if invoiceID.startswith('D') \
    else \
    json.loads(
    rs.get(
            f"{base_url}/v1/invoices/OneTime-{invoiceID}/lineitems/OneTime/BillingLineItems?size=5000",
            headers=HTTPheaders)
    .content)

    df = pd.DataFrame(response['items'])
    df['invoiceMonth'] = invoiceMonth
    cxn.query('select * from df').write_parquet(f'./DB/onetime_invoices/{invoiceMonth}.parquet') if invoiceID.startswith('G') \
    else cxn.query('select * from df').write_parquet(f'./DB/recurring_invoices/{invoiceMonth}.parquet')

    print(f"{invoiceMonth} file written to recurring_invoices folder \n" if invoiceID.startswith('D') else f"{invoiceMonth} file written to onetime_invoices folder \n")

In [ ]:
query("Select sum(totalcharges) from invoices where billingPeriodStartDate between '2023-01-13' and '2023-12-13' ") 

┌────────────────────┐
│ sum(totalcharges)  │
│       double       │
├────────────────────┤
│ 148006225.10999998 │
└────────────────────┘



In [ ]:
cxn.sql("create or replace table invoices as select * from invoicesDF") 

In [ ]:
cxn.sql('show tables')

┌───────────┐
│   name    │
│  varchar  │
├───────────┤
│ invoices  │
│ onetime   │
│ recurring │
└───────────┘

In [ ]:
cxn.sql("create or replace table invoices as select * from read_parquet('./DB//allInvoices.parquet')")
cxn.sql("create or replace table onetime as select * from read_parquet('./DB/onetime_invoices/*_2023*')")
cxn.sql("create or replace table recurring as select * from read_parquet('./DB/recurring_invoices/*')")

In [ ]:
cxn.sql("select invoicemonth, round(sum(amount),2) amount, round(sum(subtotal),2) total, round(sum(totalforcustomer),2) tfc from recurring group by invoicemonth")

┌────────────────┬────────────┬────────────┬────────────┐
│  invoiceMonth  │   amount   │   total    │    tfc     │
│    varchar     │   double   │   double   │   double   │
├────────────────┼────────────┼────────────┼────────────┤
│ August_2021    │ 4063897.77 │ 4037821.79 │ 4037821.79 │
│ January_2021   │ 2312604.26 │ 2312465.93 │ 2312465.93 │
│ March_2017     │    -162.82 │    -162.82 │    -162.82 │
│ March_2018     │   40561.66 │   40561.66 │   40561.66 │
│ March_2019     │  519927.67 │  519927.67 │  519927.67 │
│ May_2017       │    1189.76 │    1189.76 │    1189.76 │
│ November_2022  │ 1753619.04 │ 1753619.04 │ 1753619.04 │
│ September_2020 │ 1687711.69 │ 1561147.72 │ 1561147.72 │
│ September_2021 │ 4496392.83 │  4467555.7 │  4467555.7 │
│ September_2022 │ 2157058.31 │ 2149958.31 │ 2149958.31 │
│     ·          │       ·    │       ·    │       ·    │
│     ·          │       ·    │       ·    │       ·    │
│     ·          │       ·    │       ·    │       ·    │
│ April_2018  

In [ ]:
query("""select 
            invoicenumber,orderdate,
            invoicemonth,subscriptionId,subscriptionStartDate , subscriptionEndDate,skuname,
            chargeStartDate ,chargeEndDate ,
            chargeType  , unitPrice , effectiveUnitPrice , unitType , 
            quantity ,billablequantity, subtotal , taxTotal , totalForCustomer,
            termAndBillingCycle
       from
             onetime 
      where 
            productid like 'CFQ%' and customerdomainname = 'trdfin.in' and skuname = 'Microsoft 365 Business Standard' and subscriptionid = '2fa56908-b033-40b6-c8fc-a7c74f40624e'
      order by
             customerdomainname,productid, skuid,subscriptionid,orderdate,effectiveunitprice 
      limit 100""")

┌───────────────┬──────────────────────────────┬────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬─────────────────────────────────┬──────────────────────────────┬──────────────────────┬─────────────┬───────────┬────────────────────┬──────────┬──────────┬──────────────────┬──────────┬──────────┬──────────────────┬────────────────────────────────────────────────┐
│ invoiceNumber │          orderDate           │  invoiceMonth  │            subscriptionId            │ subscriptionStartDate │ subscriptionEndDate  │             skuName             │       chargeStartDate        │    chargeEndDate     │ chargeType  │ unitPrice │ effectiveUnitPrice │ unitType │ quantity │ billableQuantity │ subtotal │ taxTotal │ totalForCustomer │              termAndBillingCycle               │
│    varchar    │           varchar            │    varchar     │               varchar                │        varchar        │       varchar        │             

In [ ]:
ungrouped = cxn.query("""select customername,round(sum(amount),2) from(
      select customername,customerid,syndicationPartnerSubscriptionNumber,domainname,subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      --datediff('day',cast(chargestartdate as datetime),cast(chargeenddate as datetime) ) as noOfDays,
     chargetype,(unitprice) as unitprice,quantity,(amount) as amount
       from recurring a
      join invoices b on a.invoicenumber = b.id
      --where customername in
       --(select customername from recurring where chargetype like '%prorate%' group by customername order by count(customername) desc limit 3) 
      --and customername = 'Sukino Healthcare Solutions Pvt. Ltd' 
      --and (chargetype != 'Cycle instance prorate' or unitPrice >0)
      order by customername
      ,syndicationPartnerSubscriptionNumber,billingPeriodStartDate ,chargestartdate,chargeenddate desc
      )group by customername order by 1
            """)




In [ ]:
grouped = cxn.query("""select customername,round(sum(amount),2) from (
      select customername,customerid,syndicationPartnerSubscriptionNumber,domainname,subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      --datediff('day',cast(chargestartdate as datetime),cast(chargeenddate as datetime) ) as noOfDays,
     sum(unitprice) as unitprice,quantity,sum(amount) as amount
       from recurring a
      join invoices b on a.invoicenumber = b.id
      --where customername in
       --(select customername from recurring where chargetype like '%prorate%' group by customername order by count(customername) desc limit 3) 
      --and customername = 'Sukino Healthcare Solutions Pvt. Ltd' 
      group by customername,customerid,domainname,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      abs(unitprice),quantity,abs(amount)
      having count(1) = 1
      order by customername
      ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc
      ) group by customername order by 1
      """)

In [ ]:
 list(cxn.query("select * from ungrouped except  select * from grouped order by 1").to_df()['customerName'])


['ADITYA PARK HOTEL',
 'Altius Tech Insurance Broking Private Limited',
 'Credit Access Grameen',
 'GIBES.CO.IN',
 'M/s. Lite Auto Components Pvt Ltd',
 'MATDUN LABS INDIA PRIVATE LIMITED',
 'NADHINYA IT & DATA SERVICES PRIVATE LIMITED',
 'Prabhat Dairy Limited',
 'Ramanand Kidarnath International',
 'Tamil Nadu Newsprint and Papers Limited',
 'TestCustomer']

In [ ]:
cx = 'ADITYA PARK HOTEL'
# cx='Altius Tech Insurance Broking Private Limited'
# cx='Credit Access Grameen'
# cx='GIBES.CO.IN'
# cx='M/s. Lite Auto Components Pvt Ltd'
# cx='MATDUN LABS INDIA PRIVATE LIMITED'
# cx='NADHINYA IT & DATA SERVICES PRIVATE LIMITED'
# cx='Prabhat Dairy Limited'
# cx='Ramanand Kidarnath International'
# cx='Tamil Nadu Newsprint and Papers Limited'
cx='Y-Axis Solutions PVT Ltd'

query(f"""
      
      --select round(sum(amount),2) amount  from 
      
      (

select 
      customername,customerid,syndicationPartnerSubscriptionNumber,
      subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,date_diff('day',cast(chargestartdate as datetime),cast(chargeenddate as datetime))+1 noOFDays,
      
      chargetype,(unitprice) as unitprice,round(unitprice/noOFdays) dailyprice,
      quantity,(amount) as amount,invoiceMonth,
      case when chargetype in ('Cycle fee','Cycle instance prorate') then 1
      --when chargetype in ('Prorate fee when renew','Cancel fee','Cycle instance prorate','Prorate fees when activate','Prorate fees when cancel') then 2
      --when chargetype in ('Prorate fees when purchase','Cancel fee') then 3
      else 0
      end as rowsToBeRemoved
       from recurring a
      join invoices b on a.invoicenumber = b.id
      where
      /* customername in
       (select customername from exceptt) 
      and*/ customername = '{cx}'
      -- and subscriptionname = 'Microsoft 365 Apps for business'
      order by customername
      ,syndicationPartnerSubscriptionNumber,billingPeriodStartDate ,chargestartdate,chargeenddate desc
      
      )
--group by customername order by 2 desc

""")

┌──────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────┬────────────────────────────┬───────────┬────────────┬──────────┬───────────┬────────────────┬─────────────────┐
│       customerName       │              customerId              │ syndicationPartnerSubscriptionNumber │           subscriptionName           │ subscriptionStartDate │ subscriptionEndDate  │   chargeStartDate    │    chargeEndDate     │ noOFDays │         chargeType         │ unitprice │ dailyprice │ quantity │  amount   │  invoiceMonth  │ rowsToBeRemoved │
│         varchar          │               varchar                │               varchar                │               varchar                │        varchar        │       varchar        │       varchar        │       varchar        │  int64   │          varchar          

In [ ]:
query(f"""
      
      --select ifnull(round(sum(amount),2),0) amount  from 
      
      (
with t1 as 
(select 
      customername,customerid,substring(billingPeriodStartDate,0,8) invoiceMonth,syndicationPartnerSubscriptionNumber,
      subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,date_diff('day',cast(chargestartdate as datetime),cast(chargeenddate as datetime))+1 noOFDays,
      
      chargetype,(unitprice) as unitprice,round(unitprice/noOFdays) dailyprice,
      quantity,(amount) as amount,
      case when chargetype in ('Cycle fee','Cycle instance prorate') then 1
      when chargetype in ('Prorate fees when activate','Prorate fees when cancel') then 2
      --when chargetype in ('Prorate fees when purchase','Cancel fee') then 3
      else 0
      end as rowsToBeRemoved
       from recurring a
      join invoices b on a.invoicenumber = b.id
      where 
      /*customername in
       (select customername from exceptt) 
      and */ customername = '{cx}'
      -- and subscriptionname = 'Microsoft 365 Apps for business'
      order by customername
      ,syndicationPartnerSubscriptionNumber,billingPeriodStartDate ,chargestartdate,chargeenddate desc)
      ,
      t2 as (select 
      customername,customerid,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,max(chargetype) chargetype,chargestartdate,chargeenddate,noofdays,sum(unitprice) unitprice,quantity,sum(amount) amount
    ,rowsToBeRemoved,substring(chargestartdate,0,8) invoiceMonth

      from t1
      group by customername,customerid,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,chargestartdate,chargeenddate,noofdays,abs(unitprice),quantity,abs(amount) ,rowsToBeRemoved 
    having not (count(1) = 2 and sum(amount) = 0))
  ,
    t3 as (select 
    customername,customerid,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,max(chargetype) chargetype,min(chargestartdate) newchargestartdate
    ,chargeenddate
    ,date_add(cast(max(chargestartdate) as datetime), interval (-1) day) adjustedChargeEndDate
    --,case when substring(chargeenddate,0,8) != substring(adjustedChargeEndDate,0,8) then chargeenddate else adjustedchargeenddate end as finalChargeEndDate
    ,case when count(1) = 2 then adjustedchargeenddate else  chargeenddate end as finalChargeEndDate

    ,sum(unitprice) unitprice,quantity,sum(amount) amount,count(1) count
    from t2
    group by 
    customername,customerid,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,chargeenddate,quantity
    having not(count(1) = 2 and sum(amount) = 0))

    select 
    customername,customerid,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,newchargestartdate chargestartdate 
    --,chargeenddate,adjustedchargeenddate
    , finalchargeenddate chargeenddate , unitprice,quantity,amount
    --,count
    from t3

    order by customername,syndicationPartnerSubscriptionNumber,subscriptionname,subscriptionStartDate,
    subscriptionEndDate,newchargestartdate,chargeenddate 
      )
      --group by customername order by 1


""")

┌──────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬────────────────────┬──────────┬───────────────────┐
│       customerName       │              customerId              │ syndicationPartnerSubscriptionNumber │           subscriptionName           │ subscriptionStartDate │ subscriptionEndDate  │   chargestartdate    │    chargeenddate     │     unitprice      │ quantity │      amount       │
│         varchar          │               varchar                │               varchar                │               varchar                │        varchar        │       varchar        │       varchar        │       varchar        │       double       │  int64   │      double       │
├──────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼─────────────────────

In [ ]:
onetimetbl = cxn.sql("select * from  onetime where productid like  'CFQ%' ")



In [ ]:
query("""
select customername,count(1) from onetimetbl group by 1 order by 2 desc limit 5 

""")

┌─────────────────────────────────────────┬──────────┐
│              customerName               │ count(1) │
│                 varchar                 │  int64   │
├─────────────────────────────────────────┼──────────┤
│ TRDFIN Support Services Private Limited │      201 │
│ Annet Technologies (Gujarat)            │      184 │
│ ILENSYS TECHNOLOGIES PVT. LTD.          │      116 │
│ Mount Meru India Private Limited        │      114 │
│ Y-Axis Solutions Pvt. Ltd               │      107 │
└─────────────────────────────────────────┴──────────┘



In [ ]:
columnOrderForOneTime = """customerName,customerDomainName,invoiceNumber,orderDate,productName,skuName,subscriptionId,subscriptionStartDate,subscriptionEndDate,
                            cast(chargeStartDate as date) castedChargeStartDate,cast(chargeEndDate as date) castedChargeEndDate,chargeType,effectiveUnitPrice,billableQuantity,totalForCustomer, date_diff('day', castedchargestartdate,castedchargeenddate) noOfDays, round(effectiveunitprice/noofdays,2) dailyPrice, """
columnOrderForOneTime

"customerName,customerDomainName,invoiceNumber,orderDate,productName,skuName,subscriptionId,subscriptionStartDate,subscriptionEndDate,\n                            cast(chargeStartDate as date) castedChargeStartDate,cast(chargeEndDate as date) castedChargeEndDate,chargeType,effectiveUnitPrice,billableQuantity,totalForCustomer, date_diff('day', castedchargestartdate,castedchargeenddate) noOfDays, round(effectiveunitprice/noofdays,2) dailyPrice, "

In [5]:
cxn.close()

In [ ]:
selectOneTimeTbl = cxn.query(f'select {columnOrderForOneTime} from onetimetbl')
selectOneTimeTblColumns = selectOneTimeTbl.columns
selectOneTimeTblColumns

['customerName',
 'customerDomainName',
 'invoiceNumber',
 'orderDate',
 'productName',
 'skuName',
 'subscriptionId',
 'subscriptionStartDate',
 'subscriptionEndDate',
 'castedChargeStartDate',
 'castedChargeEndDate',
 'chargeType',
 'effectiveUnitPrice',
 'billableQuantity',
 'totalForCustomer',
 'noOfDays',
 'dailyPrice']

In [ ]:
filteredOneTimeTbl  = cxn.query(f"""select * from selectOneTimeTbl
       where customerdomainname  = 'trdfin.in' and skuname = 'Exchange Online Kiosk' and chargetype in ('new','addQuantity') and subscriptionid = '607efdba-7207-4bb0-c181-81d6db5c9adc'
                                and invoicenumber in ('G018993861','G020077203')
      """)
query('select * from filteredOneTimeTbl order by castedchargestartdate,castedchargeenddate')
filteredOneTimeTblColumns = filteredOneTimeTbl.columns
filteredOneTimeTblColumns

NameError: name 'cxn' is not defined

In [ ]:
groupByColumns = ','.join(filteredOneTimeTblColumns).replace('invoiceNumber,','').replace('orderDate,','')\
    .replace('castedChargeStartDate,','').replace('chargeType,','').replace('effectiveUnitPrice,','')\
        .replace('totalForCustomer,','').replace('noOfDays,','').replace('dailyPrice','')
groupByColumns

'customerName,customerDomainName,productName,skuName,subscriptionId,subscriptionStartDate,subscriptionEndDate,castedChargeEndDate,billableQuantity,'

In [ ]:
query(f"""select sum(adjustedtotalforcustomer) from 
      (
      select 
      {groupByColumns}
      min(cast(castedChargeStartDate as date)) adjustedChargestartdate,date_add(max(cast(castedChargeStartDate as date)), interval (-1) day) adjustedChargeEnddate
      ,round(sum(totalforcustomer),2) adjustedtotalforcustomer
      from filteredOneTimeTbl
      group by 
      {groupByColumns}
      --order by adjustedchargestartdate, adjustedchargeenddate

      )
       
      """)

┌───────────────────────────────┐
│ sum(adjustedtotalforcustomer) │
│            double             │
├───────────────────────────────┤
│                      27013.11 │
└───────────────────────────────┘



In [ ]:
query(f"""select {columnOrderForOneTime} from onetimetbl
       where invoicenumber in ('G018993861','G020077203') and subscriptionid = '607efdba-7207-4bb0-c181-81d6db5c9adc'
       and chargetype in ('new','addQuantity')
      order by subscriptionid,skuname, date_trunc('day',cast(Chargestartdate as date)),chargeenddate,billablequantity,effectiveunitprice
       """)
query("""select sum(totalforcustomer) from onetimetbl
       where invoicenumber in ('G018993861','G020077203') and subscriptionid = '607efdba-7207-4bb0-c181-81d6db5c9adc'
       and chargetype in ('new','addQuantity')
       """)

┌─────────────────────────────────────────┬────────────────────┬───────────────┬──────────────────────────────┬───────────────────────┬───────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬─────────────────────┬─────────────┬────────────────────┬──────────────────┬──────────────────┬──────────┬────────────┐
│              customerName               │ customerDomainName │ invoiceNumber │          orderDate           │      productName      │        skuName        │            subscriptionId            │ subscriptionStartDate │ subscriptionEndDate  │ castedChargeStartDate │ castedChargeEndDate │ chargeType  │ effectiveUnitPrice │ billableQuantity │ totalForCustomer │ noOfDays │ dailyPrice │
│                 varchar                 │      varchar       │    varchar    │           varchar            │        varchar        │        varchar        │               varchar                │        varchar        │

In [ ]:
query(f"""
      --select sum(adjustedamount) from 
      
      (with t1 as 
    (select 
      customername,customerid
      
      ,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      min(chargetype),
      sum(unitprice) unitprice,quantity,sum(amount) amount
      ,rowsToBeRemoved,count(1)
      from
      (

select 
      customername,customerid,syndicationPartnerSubscriptionNumber,
      subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      chargetype,(unitprice) as unitprice,
      quantity,(amount) as amount,invoiceMonth,
      case when chargetype in ('Cycle fee','Cycle instance prorate') then 1
      when chargetype in ('Prorate fee when renew','Cancel fee','Cycle instance prorate','Prorate fees when activate','Prorate fees when cancel') then 2
      --when chargetype in ('Prorate fees when purchase','Cancel fee') then 3
      else 0
      end as rowsToBeRemoved
       from recurring a
      join invoices b on a.invoicenumber = b.id
      where customername customername = '{cx}'
      --and subscriptionname = 'Microsoft 365 Apps for business'
      
      
      )
      group by customername,customerid
      
      ,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      abs(unitprice),quantity,abs(amount)
      ,rowsToBeRemoved
      --having count(1) = 1

    )
      ,
      t2 as 
      (select *,
      date_diff('day', cast(chargestartdate as datetime) ,cast(chargeenddate as datetime))+1 as noOfDays
      ,unitprice/noOfDays dailyPrice
      ,coalesce(date_add(cast(lead(chargestartdate) over(partition by customername,syndicationPartnerSubscriptionNumber order by customername
      ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc) as datetime), interval (0) day),chargeenddate)  as adjustedchargeenddate /*1 day before the next chargestartdate*/
      ,date_diff('day', cast(chargestartdate as datetime) ,cast(adjustedchargeenddate as datetime)) as adjustedNoOfDays
      ,dailyPrice * adjustedNoOfDays as adjustedUnitPrice
      ,adjustedUnitPrice * quantity as adjustedAmount
         from t1)

         select * from t2 where adjustedamount>= 0

         order by customername
      ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc)


""")

duckdb.duckdb.ParserException('Parser Error: syntax error at or near "customername"\nLINE 27:       where customername customername = \'ADITYA PARK HOTEL\'\n                                  ^')

In [ ]:
query(f"""
--select round(sum(amount),2) amount from
(with t1 as ( /* adjusting chargeenddate and removing just the nullified rows*/
      
select 
      customername,customerid,syndicationPartnerSubscriptionNumber,
      subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,date_add(cast(chargeenddate as datetime), interval 1 day) chargeenddate,
      chargetype,(unitprice) as unitprice,
      quantity,(amount) as amount,invoiceMonth,
      case when chargetype in ('Cycle fee','Cycle instance prorate') then 1
      when chargetype in ('Prorate fee when renew','Cancel fee','Cycle instance prorate','Prorate fees when activate','Prorate fees when cancel') then 2
      else 0
      end as rowsToBeRemoved
       from recurring a
      join invoices b on a.invoicenumber = b.id
      where customername  = '{cx}'
      
)
,
t2 as /* group to remove the rows*/
(select 
      customername,customerid,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,min(chargetype) chargetype,
      sum(unitprice) unitprice,quantity,sum(amount) amount,rowsToBeRemoved,count(1)
from t1
  group by customername,customerid,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,
      abs(unitprice),quantity,abs(amount)
      ,rowsToBeRemoved
       having count(1) in (1,3,5,7,9)



), t3 as

(select *,
case when chargetype in ('Prorate fees when purchase','Cancel fee') then 1 else 0 end as newrowstoberemoved
from t2)

, t4 as (select 
  customername,customerid,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,max(chargestartdate) chargestartdate,chargeenddate,min(chargetype) chargetype,
      sum(unitprice) unitprice,quantity,sum(amount) amount,count(1), newrowstoberemoved
      from t3
        group by customername,customerid,subscriptionname,syndicationPartnerSubscriptionNumber,subscriptionstartdate,subscriptionenddate,chargeenddate,
      abs(unitprice),quantity,abs(amount)
      ,newrowsToBeRemoved)
select *,
lead(chargestartdate) over(partition by customername, syndicationPartnerSubscriptionNumber order by customername
     ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc )
from t4
  order by customername
      ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc)
""")


# ,
# date_diff('day', cast(chargestartdate as datetime), cast(chargeenddate as datetime)) noOfDays
# ,unitprice/noofdays dailyprice 
# ,coalesce(cast(lead(chargestartdate) over(partition by customername,syndicationPartnerSubscriptionNumber order by customername
#       ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc) as datetime),chargeenddate)  as adjustedchargeenddate 
# ,date_diff('day', cast(chargestartdate as datetime), cast(adjustedchargeenddate as datetime)) adjustednoOfDays
# ,unitprice/adjustednoofdays adjusteddailyprice

┌───────────────────────┬──────────────────────────────────────┬───────────────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬─────────────────────┬────────────────────────────┬───────────┬──────────┬──────────┬──────────┬────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│     customerName      │              customerId              │         subscriptionName          │ syndicationPartnerSubscriptionNumber │ subscriptionStartDate │ subscriptionEndDate  │   chargestartdate    │    chargeenddate    │         chargetype         │ unitprice │ quantity │  amount  │ count(1) │ newrowstoberemoved │ lead(chargestartdate) OVER (PARTITION BY customername, syndicationPartnerSubscriptionNumber ORDER BY customername, syndicationPartnerSubscriptionNumber, c

In [ ]:
query("select customername ,count(distinct subscriptionname) from recurring group by 1 order by 2 desc")

┌──────────────────────────────────────────────────────────────────┬──────────────────────────────────┐
│                           customerName                           │ count(DISTINCT subscriptionname) │
│                             varchar                              │              int64               │
├──────────────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ AutoRABIT Software Pvt Ltd                                       │                                8 │
│ greycampus.com                                                   │                                8 │
│ Brio Technologies Pvt Ltd - POC                                  │                                7 │
│ Service Lee Technologies Pvt. Ltd.                               │                                7 │
│ Sansera Engineering                                              │                                6 │
│ Ignitarium Technology Solutions Private Limited               

In [ ]:
query("""select * from recurring where customername = 'AutoRABIT Software Pvt Ltd' 
      and  subscriptionname in ('Office 365 Business Premium India','Office 365 Business Essentials India')
      order by customername
      ,syndicationPartnerSubscriptionNumber ,chargestartdate,chargeenddate desc,unitprice desc""")

┌──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬────────────┬────────────────────┬───────────────┬──────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────┬──────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬────────────────────────────┬───────────┬──────────┬───────────┬────────────────────┬───────────┬────────┬──────────────────┬──────────┬─────────────────────┬─────────────────┬──────────────────────────────────────┬────────────────┐
│              partnerId               │              customerId              │        customerName        │  mpnId  │ tier2MpnId │      orderId       │ invoiceNumber │  subscriptionId  │ syndicationPartnerSubscription

In [ ]:
query("""
      select 
      --*,
      customername, customerid,productid,substring(productname,0,15) productname,skuid,substring(skuname,0,15) skuname,subscriptionid,
      subscriptionStartDate ,subscriptionEndDate,orderdate,chargeStartDate,chargeEndDate,
      chargetype,unitPrice ,quantity,billableQuantity ,effectiveUnitPrice,subtotal,taxTotal,totalforcustomer,referenceId,termandbillingcycle,invoicemonth        
      from onetime 
      where customername in ('TRDFIN Support Services Private Limited','Annet Technologies (Gujarat)')
      and  productid like 'CFQ%'
      and referenceid in ('2bf1261c-dc09-4bc0-9d73-ecc41783c9c8','26f0548f-7e53-41ab-a430-e783e7d1072b','abb2a6c1-7612-4d37-90e2-33fa8b9f74ec','5b67f605-6078-4878-ba1e-fab43173e299'
      ,'1019856a-d88a-43d6-b3ff-2c419a197901','5b1634df-d1ee-4afa-a989-552e6812486c')
      order by referenceid,
       customername, customerid,
      subscriptiondescription,productname,skuname,
      subscriptionStartDate ,subscriptionEndDate,orderdate,chargeStartDate,chargeEndDate
      """)

┌──────────────────────────────┬──────────────────────────────────────┬──────────────┬────────────────┬─────────┬────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────────────┬──────────────────────────────┬──────────────────────┬──────────────┬───────────┬──────────┬──────────────────┬────────────────────┬───────────┬───────────┬──────────────────┬──────────────────────────────────────┬──────────────────────────────────────────┬──────────────┐
│         customerName         │              customerId              │  productId   │  productname   │  skuId  │    skuname     │            subscriptionId            │ subscriptionStartDate │ subscriptionEndDate  │          orderDate           │       chargeStartDate        │    chargeEndDate     │  chargeType  │ unitPrice │ quantity │ billableQuantity │ effectiveUnitPrice │ subtotal  │ taxTotal  │ totalForCustomer │             referenceId              │           termAndB

In [ ]:
query("""
      select 
      --*,
      productid,substring(productname,0,15) productname,skuid,substring(skuname,0,15) skuname,subscriptionid,
      subscriptionStartDate ,subscriptionEndDate,orderdate,chargeStartDate,chargeEndDate,
      chargetype,unitPrice ,quantity,billableQuantity ,effectiveUnitPrice,subtotal,taxTotal,totalforcustomer,referenceId,termandbillingcycle,invoicemonth        
      from onetime 
      where customername in ('TRDFIN Support Services Private Limited','Annet Technologies (Gujarat)')
      and  productid like 'CFQ%'
      and referenceid in ('2bf1261c-dc09-4bc0-9d73-ecc41783c9c8','26f0548f-7e53-41ab-a430-e783e7d1072b','abb2a6c1-7612-4d37-90e2-33fa8b9f74ec','5b67f605-6078-4878-ba1e-fab43173e299'
      ,'1019856a-d88a-43d6-b3ff-2c419a197901','5b1634df-d1ee-4afa-a989-552e6812486c')
      order by referenceid,
       customername, customerid,
      subscriptiondescription,productname,skuname,
      subscriptionStartDate ,subscriptionEndDate,orderdate,chargeStartDate,chargeEndDate
      """)

┌──────────────┬────────────────┬─────────┬────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────────────┬──────────────────────────────┬──────────────────────┬──────────────┬───────────┬──────────┬──────────────────┬────────────────────┬───────────┬───────────┬──────────────────┬──────────────────────────────────────┬──────────────────────────────────────────┬──────────────┐
│  productId   │  productname   │  skuId  │    skuname     │            subscriptionId            │ subscriptionStartDate │ subscriptionEndDate  │          orderDate           │       chargeStartDate        │    chargeEndDate     │  chargeType  │ unitPrice │ quantity │ billableQuantity │ effectiveUnitPrice │ subtotal  │ taxTotal  │ totalForCustomer │             referenceId              │           termAndBillingCycle            │ invoiceMonth │
│   varchar    │    varchar     │ varchar │    varchar     │               varchar                │ 

In [ ]:
'''this
is a multime
 comment'''

'this\nis a multime\n comment'

In [ ]:
cx='Prabhat Dairy Limited'
# cx='Ramanand Kidarnath International'
# cx='Tamil Nadu Newsprint and Papers Limited'
# cx='TestCustomer'

query(f"""
      
      --select round(sum(amount),2) amount  from 
      
      (

select 
      customername,customerid,syndicationPartnerSubscriptionNumber,
      subscriptionname,subscriptionstartdate,subscriptionenddate,chargestartdate,chargeenddate,date_diff('day',cast(chargestartdate as datetime),cast(chargeenddate as datetime))+1 noOFDays,
      
      chargetype,(unitprice) as unitprice,unitprice/noOFdays dailyprice,
      quantity,(amount) as amount,invoiceMonth,
      case when chargetype in ('Cycle fee','Cycle instance prorate') then 1
      when chargetype in ('Prorate fee when renew','Cancel fee','Cycle instance prorate','Prorate fees when activate','Prorate fees when cancel') then 2
      --when chargetype in ('Prorate fees when purchase','Cancel fee') then 3
      else 0
      end as rowsToBeRemoved
       from recurring a
      join invoices b on a.invoicenumber = b.id
      where customername in
       (select customername from exceptt) 
      and customername = '{cx}'
      -- and subscriptionname = 'Microsoft 365 Apps for business'
      order by customername
      ,syndicationPartnerSubscriptionNumber,billingPeriodStartDate ,chargestartdate,chargeenddate desc
      
      )


""")

┌──────────────┬────────────┬──────────────────────────────────────┬──────────────────┬───────────────────────┬─────────────────────┬─────────────────┬───────────────┬──────────┬────────────┬───────────┬────────────┬──────────┬────────┬──────────────┬─────────────────┐
│ customerName │ customerId │ syndicationPartnerSubscriptionNumber │ subscriptionName │ subscriptionStartDate │ subscriptionEndDate │ chargeStartDate │ chargeEndDate │ noOFDays │ chargeType │ unitprice │ dailyprice │ quantity │ amount │ invoiceMonth │ rowsToBeRemoved │
│   varchar    │  varchar   │               varchar                │     varchar      │        varchar        │       varchar       │     varchar     │    varchar    │  int64   │  varchar   │  double   │   double   │  int64   │ double │   varchar    │      int32      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:



f or index,row in invoicesDF.iterrows():
    
    print('Starting',row['ID'],row['invoiceMonth'],row['invoiceType'] )
    
    response = \
    json.loads(
    rs.get(
            f"{base_url}/v1/invoices/Recurring-{row['ID']}/lineitems/Office/BillingLineItems",
            headers=HTTPheaders)
    .content) if row['invoiceType'] == 'Recurring' \
    else \
    json.loads(
    rs.get(
            f"{base_url}/v1/invoices/OneTime-{row['ID']}/lineitems/OneTime/BillingLineItems?size=5000",
            headers=HTTPheaders)
    .content)

    df = pd.DataFrame(response['items'])
    df['invoiceMonth'] = row['invoiceMonth']
    cxn.query('select * from df').write_parquet(f'./DB/onetime_invoices/{row["invoiceMonth"]}.parquet') if row['invoiceType'] == 'OneTime'\
    else cxn.query('select * from df').write_parquet(f'./DB/recurring_invoices/{row["invoiceMonth"]}.parquet')

    print(f"{row['invoiceMonth']} file written to recurring \n" if row['invoiceType'] == 'Recurring' else f"{row['invoiceMonth']} file written to OneTime \n")


Starting G036230777 December_2023 OneTime
December_2023 file written to OneTime 

Starting G033956933 November_2023 OneTime
November_2023 file written to OneTime 

Starting G031809747 October_2023 OneTime
October_2023 file written to OneTime 

Starting G029930588 September_2023 OneTime
September_2023 file written to OneTime 

Starting G028190160 August_2023 OneTime
August_2023 file written to OneTime 

Starting G026628959 July_2023 OneTime
July_2023 file written to OneTime 

Starting G025172256 June_2023 OneTime
June_2023 file written to OneTime 

Starting G023804626 May_2023 OneTime
May_2023 file written to OneTime 

Starting G022498969 April_2023 OneTime
April_2023 file written to OneTime 

Starting G021249609 March_2023 OneTime
March_2023 file written to OneTime 

Starting G020077203 February_2023 OneTime
February_2023 file written to OneTime 

Starting G018993861 January_2023 OneTime
January_2023 file written to OneTime 

Starting G017943686 December_2022 OneTime
December_2022 file

In [ ]:
legacyOfficeURL = '/v1/invoices/Recurring-D080003O1Z/lineitems/Office/BillingLineItems'

response = json.loads(
    rs.get(
            f"{base_url}{legacyOfficeURL}",
            headers=HTTPheaders)
    .content) if rs.get(
            f"{base_url}{legacyOfficeURL}",
            headers=HTTPheaders).status_code == 200 else 'no data'


In [ ]:
legacyAzureBillingURL =   '/v1/invoices/Recurring-D080003O1Z/lineitems/Azure/BillingLineItems'

response = json.loads(
    rs.get(
            f"{base_url}{legacyAzureBillingURL}",
            headers=HTTPheaders)
    .content) if rs.get(
            f"{base_url}{legacyAzureBillingURL}",
            headers=HTTPheaders).status_code == 200 else 'no data'


In [ ]:

if response != 'no data':
        legAzureBillingDF = pd.DataFrame(response['items'])
        query("select * from legAzureBillingDF")
else:
    print('no data')

In [ ]:
legacyAzureUsageURL = '/v1/invoices/Recurring-D080003O1Z/lineitems/Azure/UsageLineItems'

response = json.loads(
    rs.get(
            f"{base_url}{legacyAzureUsageURL}",
            headers=HTTPheaders)
    .content) if rs.get(
            f"{base_url}{legacyAzureUsageURL}",
            headers=HTTPheaders).status_code == 200 else 'no data'

response


In [ ]:

if response != 'no data':
        legAzureUsageDF = pd.DataFrame(response['items'])
        query("select * from legAzureUsageDF")
else:
    print('no data')

#new commerce

In [ ]:
newCommerceOneTimeBillingURL = '/v1/invoices/OneTime-G038562576/lineitems/OneTime/BillingLineItems?size=5000'

response = json.loads(
    rs.get(
            f"{base_url}{newCommerceOneTimeBillingURL}",
            headers=HTTPheaders)
    .content) if rs.get(
            f"{base_url}{newCommerceOneTimeBillingURL}",
            headers=HTTPheaders).status_code == 200 else 'no data' # if response status code = 200
response


{'totalCount': 3216,
 'items': [{'partnerId': '6e75cca6-47f0-47a3-a928-9d5315750bd9',
   'customerId': '86fb359e-1360-4ab3-b90d-2a68e8c007b9',
   'customerName': 'Data Semantics Pvt Ltd',
   'customerDomainName': 'DSglobal.onmicrosoft.com',
   'customerCountry': 'IN',
   'invoiceNumber': 'G038562576',
   'mpnId': '2240443',
   'resellerMpnId': 0,
   'orderId': 'i865EsrwBdGpAO_xbJyBQLW1kiHa6n5-1',
   'orderDate': '2024-01-02T09:38:43.4669373Z',
   'productId': 'DZH318Z09HH9',
   'skuId': '0016',
   'availabilityId': 'DZH318Z0B3RM',
   'productName': 'Twilio SendGrid',
   'skuName': 'Essentials 50K',
   'productQualifiers': [],
   'chargeType': 'renew',
   'unitPrice': 1530.01357,
   'effectiveUnitPrice': 1530.01,
   'unitType': '',
   'quantity': 1,
   'subtotal': 1530.01,
   'taxTotal': 275.4,
   'totalForCustomer': 1805.41,
   'currency': 'INR',
   'publisherName': 'SendGrid',
   'publisherId': '18197210',
   'subscriptionDescription': 'account-receivable',
   'subscriptionId': '2fec0

In [ ]:


newCommerceOneTimeBillingDF = pd.DataFrame(response['items'])


In [ ]:
cxn.query("""select 
            customername,customerid,customerdomainname,max(billablequantity) maxQuantity 
      from 
            newCommerceOneTimeBillingDF 
      where
           productid like 'CFQ%' 
      group by 
            customername,customerid,customerdomainname 
      order by 
            maxQuantity desc """).to_csv('OfficeInvoice_January2024.csv')

In [ ]:
cxn.close()